In [1]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-h5f8bwmn
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-h5f8bwmn
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit 62e41ae76b4b9724452ed59421c19e1caebf80a4
  Created wheel for JUSTFAIR-Toolbox: filename=JUSTFAIR_Toolbox-0.1.dev0-py3-none-any.whl size=1981 sha256=f2d3ed180a68836afdf85803e85b94f76dac80b928327e190d50a56c9b22deb3
  Stored in directory: /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-ephem-wheel-cache-8k8vnk_z/wheels/9f/1e/47/4d44d5f699854fb0b65396a893e7d2e33825e8c788e91bd1ba
Successfully built JUSTFAIR-Toolbox


In [3]:
import JUSTFAIR_Tools as jt